In [ ]:
from bokeh.plotting import gmap, output_notebook, show
output_notebook()

# add project source directory to python path:
import sys
path = '/home/greg/src/missile-map/server'
if path not in sys.path:
    sys.path.append(path)

# check Google Maps API key:
import os
if 'MAPS_API_KEY' not in os.environ:
    raise Exception('MAPS_API_KEY environment variable is required to render Google maps (https://developers.google.com/maps/documentation/javascript/get-api-key)')

# WARNING:
# DO NOT ADD notebooks with "gmap" to Git - it leaks the key

In [ ]:
from geopy import Point
from simulator import Simulator, Observer, random_location
from missilemap import Target
import random

r = random.Random(12345)

# generate targets:
path = [
    Point(45.361285195897885, 33.90794799044153),
    Point(47.487079766379715, 33.081535775384715),
    Point(49.47728424495352, 27.901909920451157),
    Point(49.83269083681804, 24.09401307480089)
]

path2 = [
    Point(45.361285195897885, 33.90794799044153),
    Point(49.3, 33.081535775384715),
    Point(50.3, 28.3)
]

# generate observer locations:
DEFAULT_RADIUS = 5000
observers = []

for p_from, p_to in zip(path[:-1], path[1:]):
    for _ in range(10):
        observers.append(Observer(location=random_location(p_from, p_to, DEFAULT_RADIUS, random=r), radius=DEFAULT_RADIUS))
            
for p_from, p_to in zip(path2[:-1], path2[1:]):
    for _ in range(10):
        observers.append(Observer(location=random_location(p_from, p_to, DEFAULT_RADIUS, random=r), radius=DEFAULT_RADIUS))

sim = Simulator(targets=[
    Target(path=path, start_time=0),
    Target(path=path2, start_time=30),
], observers=observers, random=r)

show(sim.render())


In [ ]:
import sys
# import importlib

# importlib.reload(sys.modules['missilemap.analysis'])
# importlib.reload(sys.modules['simulator.plotting'])

from missilemap.analysis import expectation_maximization

proj = expectation_maximization(
    sightings=sim.sightings,
    n_segments=5,
    iterations=20
)

from simulator.plotting import render_path

gmap = sim.render()

for p in proj:
    render_path(gmap, p.path, line_color='magenta', line_width=6, line_alpha=0.5)
    
show(gmap)


## Using Gausian Mixture Algorithm

In [ ]:
points = numpy.array([
    [s.timestamp, s.latitude, s.longitude] for s in sim.sightings
])


from sklearn.mixture import GaussianMixture

m = GaussianMixture(n_components=3)
res = m.fit_predict(points)
res


In [ ]:
sim.estimated

In [ ]:
from missilemap.analysis import sightings_to_targets

sim.sightings[16]

In [ ]:
sim.estimated[1].at_time(2716), sim.estimated[2].at_time(2716)

In [ ]:
from geopy.distance import distance
from missilemap.utils import get_bearing

target = sim.estimated[2]

distance(meters=(2716 - target.end_time) * target.speed).destination(target.path[-1], bearing=get_bearing(target.path[-2], target.path[-1]))


In [ ]:
from missilemap.utils import interpolate

In [ ]:
alpha = 1 + (2716 - target.end_time) / (target.end_time - target.start_time)

# (target.path[-1].latitude * alpha + (1 - alpha) * target.path[-2].latitude, target.path[-1].longitude * alpha + (1 - alpha) * target.path[-2].longitude)
interpolate(target.path[-2], target.path[-1], alpha)